# Tests of Buffered Iterator `link` method

### Imports

In [1]:
from pprint import pprint
import random
from buffered_iterator import BufferedIterator, BufferedIteratorEOF
from buffered_iterator import BufferedIteratorValueError


### Logging

In [2]:
import logging
logging.basicConfig(format='%(name)-20s - %(levelname)s: %(message)s')
#logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger('Line Count Tests')
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)


### Basic Parameters

In [3]:
buffer_size = 5
num_items = 15


### Display Functions

In [4]:
def buffered_iterator_compare(iter1, iter2, 
                              label1='From Iterator', label2='To Iterator'):
    
    row_template = ''.join([
        '\t{Label:<20s}',
        '{first_iter_item:<30s}',
        '{second_iter_item:<30s}\n'
        ])

    iterator_compare_str = ''.join([
        row_template.format(Label='', 
                            first_iter_item=label1, 
                            second_iter_item=label2),
        row_template.format(Label='Previous Items', 
                            first_iter_item=str(list(iter1.previous_items)),
                            second_iter_item=str(list(iter2.previous_items))),
        row_template.format(Label='Future Items', 
                            first_iter_item=str(list(iter1.future_items)),
                            second_iter_item=str(list(iter2.future_items))),
        row_template.format(Label='Item Count', 
                            first_iter_item=str(iter1.item_count),
                            second_iter_item=str(iter2.item_count)),
        row_template.format(Label='Step Back', 
                            first_iter_item=str(iter1._step_back),
                            second_iter_item=str(iter2._step_back)),
        row_template.format(Label='Buffer Size', 
                            first_iter_item=str(iter1.buffer_size),
                            second_iter_item=str(iter2.buffer_size))
        ])
    
    return iterator_compare_str

## Tests

### Validate minimum bluffer size is 1

In [5]:
try:
    test_iter = BufferedIterator(range(5), buffer_size=0)
except BufferedIteratorValueError: 
    print('pass')

pass


### BufferedIterator Conditions
- Two BufferedIterators
    - __From__ BufferedIterator
    - __To__ BufferedIterator
<br>
- Buffer Status (Previous & Future)
    - empty
    - full
    - partial
<br>
- Iterator Status
    - Not started
    - In progress
    - Closed
<br>
- Buffer Size
    - Same
    - Target larger
    - Target smaller 
		


#### deque copying 

|Condition|From BufferedIterator|To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|full|full|
|future deque|empty|empty|
|iterator status|in progress|in progress|
|buffer size|Same|Same| 

In [6]:
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size)

to_iter = BufferedIterator((i for i in range(num_items+1, num_items*2)), 
                              buffer_size=buffer_size)

for i in range(buffer_size+1):
    next(from_iter)
    next(to_iter)
    
for i in range(buffer_size+1):
    next(from_iter)

print(buffered_iterator_compare(from_iter, to_iter))

to_iter.link(from_iter)
print(buffered_iterator_compare(from_iter, to_iter))

from_iter.previous_items == to_iter.previous_items
from_iter.future_items == to_iter.future_items 

	                    From Iterator                 To Iterator                   
	Previous Items      [7, 8, 9, 10, 11]             [17, 18, 19, 20, 21]          
	Future Items        []                            []                            
	Item Count          12                            6                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             

	                    From Iterator                 To Iterator                   
	Previous Items      [7, 8, 9, 10, 11]             [7, 8, 9, 10, 11]             
	Future Items        []                            []                            
	Item Count          12                            12                            
	Step Back           0                             0                             
	Buffer Size         5                             5                             



True


|Condition|From BufferedIterator |To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|empty|empty|
|future deque|full |full |
|iterator status |in progress |in progress |
|buffer size |Same|Same |
#


In [7]:
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size)

to_iter = BufferedIterator((i for i in range(num_items+1, num_items*2)), 
                              buffer_size=buffer_size)

for i in range(buffer_size + 1):
    next(from_iter)
    next(to_iter)
from_iter.backup(buffer_size)
to_iter.backup(buffer_size)

print(buffered_iterator_compare(from_iter, to_iter))

to_iter.link(from_iter)
print(buffered_iterator_compare(from_iter, to_iter))

from_iter.previous_items == to_iter.previous_items
from_iter.future_items == to_iter.future_items 

	                    From Iterator                 To Iterator                   
	Previous Items      []                            []                            
	Future Items        [1, 2, 3, 4, 5]               [17, 18, 19, 20, 21]          
	Item Count          1                             1                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             

	                    From Iterator                 To Iterator                   
	Previous Items      []                            []                            
	Future Items        [1, 2, 3, 4, 5]               []                            
	Item Count          1                             1                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             



False

In [8]:
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size)

to_iter = BufferedIterator((i for i in range(num_items+1, num_items*2)), 
                              buffer_size=buffer_size)

for i in range(buffer_size + 1):
    next(from_iter)
    next(to_iter)
from_iter.backup(buffer_size)
to_iter.backup(buffer_size)

print(buffered_iterator_compare(from_iter, to_iter))

to_iter.link(from_iter, include_future_items=True)
print(buffered_iterator_compare(from_iter, to_iter))

from_iter.previous_items == to_iter.previous_items
from_iter.future_items == to_iter.future_items 

	                    From Iterator                 To Iterator                   
	Previous Items      []                            []                            
	Future Items        [1, 2, 3, 4, 5]               [17, 18, 19, 20, 21]          
	Item Count          1                             1                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             

	                    From Iterator                 To Iterator                   
	Previous Items      []                            []                            
	Future Items        [1, 2, 3, 4, 5]               [1, 2, 3, 4, 5]               
	Item Count          1                             1                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             



True


|Condition|From BufferedIterator |To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|empty|full|
|future deque|empty |empty |
|iterator status |in progress |in progress |
|buffer size |Same|Same|	


In [9]:
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size)

to_iter = BufferedIterator((i for i in range(num_items+1, num_items*2)), 
                              buffer_size=buffer_size)

for i in range(buffer_size + 1):
    next(to_iter)

print(buffered_iterator_compare(from_iter, to_iter))


to_iter.link(from_iter)
print(buffered_iterator_compare(from_iter, to_iter))

from_iter.previous_items == to_iter.previous_items
from_iter.future_items == to_iter.future_items 

	                    From Iterator                 To Iterator                   
	Previous Items      []                            [17, 18, 19, 20, 21]          
	Future Items        []                            []                            
	Item Count          0                             6                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             

	                    From Iterator                 To Iterator                   
	Previous Items      []                            []                            
	Future Items        []                            []                            
	Item Count          0                             0                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             



True


|Condition|From BufferedIterator |To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|empty|empty|
|future deque|empty |full |
|iterator status |in progress |in progress |
|buffer size |Same|Same|	


In [10]:
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size)

to_iter = BufferedIterator((i for i in range(num_items+1, num_items*2)), 
                              buffer_size=buffer_size)

for i in range(buffer_size + 1):
    next(to_iter)
to_iter.backup(buffer_size)

print(buffered_iterator_compare(from_iter, to_iter))

to_iter.link(from_iter)
print(buffered_iterator_compare(from_iter, to_iter))

from_iter.previous_items == to_iter.previous_items
from_iter.future_items == to_iter.future_items 

	                    From Iterator                 To Iterator                   
	Previous Items      []                            []                            
	Future Items        []                            [17, 18, 19, 20, 21]          
	Item Count          0                             1                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             

	                    From Iterator                 To Iterator                   
	Previous Items      []                            []                            
	Future Items        []                            []                            
	Item Count          0                             0                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             



True


|Condition|From BufferedIterator |To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|empty|full|
|future deque|full |empty |
|iterator status |in progress |in progress |
|buffer size |Same|Same|	


In [11]:
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size)

to_iter = BufferedIterator((i for i in range(num_items+1, num_items*2)), 
                              buffer_size=buffer_size)

for i in range(buffer_size + 1):
    next(from_iter)
    next(to_iter)
from_iter.backup(buffer_size)

print(buffered_iterator_compare(from_iter, to_iter))

to_iter.link(from_iter)
print(buffered_iterator_compare(from_iter, to_iter))

from_iter.previous_items == to_iter.previous_items
from_iter.future_items == to_iter.future_items 

	                    From Iterator                 To Iterator                   
	Previous Items      []                            [17, 18, 19, 20, 21]          
	Future Items        [1, 2, 3, 4, 5]               []                            
	Item Count          1                             6                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             

	                    From Iterator                 To Iterator                   
	Previous Items      []                            []                            
	Future Items        [1, 2, 3, 4, 5]               []                            
	Item Count          1                             1                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             



False

#### partial deque

|Condition|From BufferedIterator|To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|partial|empty|
|future deque|partial|empty|
|iterator status|in progress|in progress|
|buffer size|Same|Same| 

In [12]:
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size)

to_iter = BufferedIterator((i for i in range(num_items+1, num_items*2)), 
                              buffer_size=buffer_size)

fwd_count = random.randint(2, buffer_size-1)
back_count = random.randint(1, fwd_count-1)

for i in range(fwd_count):
    next(from_iter)
from_iter.backup(back_count)

print(buffered_iterator_compare(from_iter, to_iter))

to_iter.link(from_iter)
print(buffered_iterator_compare(from_iter, to_iter))

from_iter.previous_items == to_iter.previous_items
from_iter.future_items == to_iter.future_items 

	                    From Iterator                 To Iterator                   
	Previous Items      [0, 1]                        []                            
	Future Items        [2, 3]                        []                            
	Item Count          2                             0                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             

	                    From Iterator                 To Iterator                   
	Previous Items      [0, 1]                        [0, 1]                        
	Future Items        [2, 3]                        []                            
	Item Count          2                             2                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             



False


|Condition|From BufferedIterator|To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|partial|partial|
|future deque|partial|partial|
|iterator status|in progress|in progress|
|buffer size|Same|Same| 

In [13]:
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size)

to_iter = BufferedIterator((i for i in range(num_items+1, num_items*2)), 
                              buffer_size=buffer_size)

fwd_count = random.randint(2, buffer_size-1)
back_count = random.randint(1, fwd_count-1)

for i in range(fwd_count):
    next(from_iter)
    next(to_iter)
from_iter.backup(back_count)
to_iter.backup(back_count)

print(buffered_iterator_compare(from_iter, to_iter))

to_iter.link(from_iter)
print(buffered_iterator_compare(from_iter, to_iter))

from_iter.previous_items == to_iter.previous_items
from_iter.future_items == to_iter.future_items 

	                    From Iterator                 To Iterator                   
	Previous Items      [0, 1]                        [16, 17]                      
	Future Items        [2, 3]                        [18, 19]                      
	Item Count          2                             2                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             

	                    From Iterator                 To Iterator                   
	Previous Items      [0, 1]                        [0, 1]                        
	Future Items        [2, 3]                        []                            
	Item Count          2                             2                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             



False


|Condition|From BufferedIterator |To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|partial|full|
|future deque|partial |full |
|iterator status |in progress |in progress |
|buffer size |Same |Same |


In [14]:
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size)

to_iter = BufferedIterator((i for i in range(num_items+1, num_items*2)), 
                              buffer_size=buffer_size)

fwd_count = random.randint(2, buffer_size-1)
back_count = random.randint(1, fwd_count-1)

for i in range(fwd_count):
    next(from_iter)
from_iter.backup(back_count)

for i in range(buffer_size + 1):
    next(to_iter)
to_iter.backup(buffer_size)

print(buffered_iterator_compare(from_iter, to_iter))

to_iter.link(from_iter)
print(buffered_iterator_compare(from_iter, to_iter))

from_iter.previous_items == to_iter.previous_items
from_iter.future_items == to_iter.future_items 

	                    From Iterator                 To Iterator                   
	Previous Items      [0, 1]                        []                            
	Future Items        [2]                           [17, 18, 19, 20, 21]          
	Item Count          2                             1                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             

	                    From Iterator                 To Iterator                   
	Previous Items      [0, 1]                        [0, 1]                        
	Future Items        [2]                           []                            
	Item Count          2                             2                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             



False

#### _Different Buffer Size_
		
|Condition|From BufferedIterator|To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|full|full|
|future deque|empty|empty|
|iterator status|in progress|in progress|
|buffer size||Target larger| 

In [15]:
num_items = 9
buffer_size_from = 3
buffer_size_to = 5

from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_from)

to_iter = BufferedIterator((i for i in range(num_items+1, num_items*2)), 
                              buffer_size=buffer_size_to)

fwd_count_from = buffer_size_from + 1

for i in range(fwd_count_from):
    next(from_iter)

fwd_count_to = buffer_size_to + 1

for i in range(buffer_size_to + 1):
    next(to_iter)

print(buffered_iterator_compare(from_iter, to_iter))

to_iter.link(from_iter)
print(buffered_iterator_compare(from_iter, to_iter))

from_iter.previous_items == to_iter.previous_items
from_iter.future_items == to_iter.future_items 

	                    From Iterator                 To Iterator                   
	Previous Items      [1, 2, 3]                     [11, 12, 13, 14, 15]          
	Future Items        []                            []                            
	Item Count          4                             6                             
	Step Back           0                             0                             
	Buffer Size         3                             5                             

	                    From Iterator                 To Iterator                   
	Previous Items      [1, 2, 3]                     [1, 2, 3]                     
	Future Items        []                            []                            
	Item Count          4                             4                             
	Step Back           0                             0                             
	Buffer Size         3                             5                             



True

|Condition|From BufferedIterator |To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|full |full |
|future deque|empty|empty|
|iterator status |in progress |in progress |
|buffer size ||Target smaller|Target smaller|

**Note:** Since buffer sizes are different the _TO_ Previous Items will only contain the last _n_ items from `from_iter` , where _n_ is `buffer_size_to`.

In [16]:
num_items = 9
buffer_size_from = 5
buffer_size_to = 3

from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_from)

to_iter = BufferedIterator((i for i in range(num_items+1, num_items*2)), 
                              buffer_size=buffer_size_to)

fwd_count_from = buffer_size_from + 1

for i in range(fwd_count_from):
    next(from_iter)

fwd_count_to = buffer_size_to + 1

for i in range(buffer_size_to + 1):
    next(to_iter)

print(buffered_iterator_compare(from_iter, to_iter))

to_iter.link(from_iter)
print(buffered_iterator_compare(from_iter, to_iter))

list(from_iter.previous_items)[-buffer_size_to:] == list(to_iter.previous_items)
#from_iter.future_items == to_iter.future_items 

	                    From Iterator                 To Iterator                   
	Previous Items      [1, 2, 3, 4, 5]               [11, 12, 13]                  
	Future Items        []                            []                            
	Item Count          6                             4                             
	Step Back           0                             0                             
	Buffer Size         5                             3                             

	                    From Iterator                 To Iterator                   
	Previous Items      [1, 2, 3, 4, 5]               [3, 4, 5]                     
	Future Items        []                            []                            
	Item Count          6                             6                             
	Step Back           0                             0                             
	Buffer Size         5                             3                             



True

|Condition|From BufferedIterator |To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|empty|empty|
|future deque|full |full |
|iterator status |in progress |in progress |
|buffer size ||Target larger||

In [17]:
num_items = 9
buffer_size_from = 3
buffer_size_to = 5
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_from)

to_iter = BufferedIterator((i for i in range(num_items+1, num_items*2)), 
                              buffer_size=buffer_size_to)

fwd_count_from = buffer_size_from + 1
back_count_from = buffer_size_from

for i in range(fwd_count_from):
    next(from_iter)
from_iter.backup(back_count_from)

fwd_count_to = buffer_size_to + 1
back_count_to = buffer_size_to

for i in range(buffer_size_to + 1):
    next(to_iter)
to_iter.backup(buffer_size_to)

print(buffered_iterator_compare(from_iter, to_iter))

to_iter.link(from_iter)
print(buffered_iterator_compare(from_iter, to_iter))

from_iter.previous_items == to_iter.previous_items
from_iter.future_items == to_iter.future_items 

	                    From Iterator                 To Iterator                   
	Previous Items      []                            []                            
	Future Items        [1, 2, 3]                     [11, 12, 13, 14, 15]          
	Item Count          1                             1                             
	Step Back           0                             0                             
	Buffer Size         3                             5                             

	                    From Iterator                 To Iterator                   
	Previous Items      []                            []                            
	Future Items        [1, 2, 3]                     []                            
	Item Count          1                             1                             
	Step Back           0                             0                             
	Buffer Size         3                             5                             



False

|Condition|From BufferedIterator |To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|empty|empty|
|future deque|full |full |
|iterator status |in progress |in progress |
|buffer size ||Target smaller|Target smaller|

**Note:** Since buffer sizes are different the _TO_ Previous Items will only contain the last _n_ items from `from_iter` , where _n_ is `buffer_size_to`.

In [18]:
num_items = 9
buffer_size_from = 5
buffer_size_to = 3
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_from)

to_iter = BufferedIterator((i for i in range(num_items+1, num_items*2)), 
                              buffer_size=buffer_size_to)

fwd_count_from = buffer_size_from + 1
back_count_from = buffer_size_from

for i in range(fwd_count_from):
    next(from_iter)
from_iter.backup(back_count_from)

fwd_count_to = buffer_size_to + 1
back_count_to = buffer_size_to

for i in range(buffer_size_to + 1):
    next(to_iter)
to_iter.backup(buffer_size_to)

print(buffered_iterator_compare(from_iter, to_iter))

to_iter.link(from_iter)
print(buffered_iterator_compare(from_iter, to_iter))

from_iter.previous_items == to_iter.previous_items
list(from_iter.future_items)[-buffer_size_to:] == list(to_iter.future_items)

	                    From Iterator                 To Iterator                   
	Previous Items      []                            []                            
	Future Items        [1, 2, 3, 4, 5]               [11, 12, 13]                  
	Item Count          1                             1                             
	Step Back           0                             0                             
	Buffer Size         5                             3                             

	                    From Iterator                 To Iterator                   
	Previous Items      []                            []                            
	Future Items        [1, 2, 3, 4, 5]               []                            
	Item Count          1                             1                             
	Step Back           0                             0                             
	Buffer Size         5                             3                             



False

|Condition|From BufferedIterator |To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|empty|full|
|future deque|empty |empty |
|iterator status |in progress |in progress |
|buffer size ||Target smaller|

In [19]:
num_items = 10
buffer_size_from = 5
buffer_size_to = 3
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_from)

to_iter = BufferedIterator((i for i in range(num_items+1, num_items*2)), 
                              buffer_size=buffer_size_to)

fwd_count_to = buffer_size_to + 1
back_count_to = buffer_size_to

for i in range(buffer_size_to + 1):
    next(to_iter)
to_iter.backup(buffer_size_to)

print(buffered_iterator_compare(from_iter, to_iter))

to_iter.link(from_iter)
print(buffered_iterator_compare(from_iter, to_iter))

from_iter.previous_items == to_iter.previous_items
from_iter.future_items == to_iter.future_items 

	                    From Iterator                 To Iterator                   
	Previous Items      []                            []                            
	Future Items        []                            [12, 13, 14]                  
	Item Count          0                             1                             
	Step Back           0                             0                             
	Buffer Size         5                             3                             

	                    From Iterator                 To Iterator                   
	Previous Items      []                            []                            
	Future Items        []                            []                            
	Item Count          0                             0                             
	Step Back           0                             0                             
	Buffer Size         5                             3                             



True

|Condition|From BufferedIterator|To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|partial|empty|
|future deque|partial|empty|
|iterator status|in progress|in progress|
|buffer size ||Target smaller|

In [20]:
num_items = 10
buffer_size_from = 5
buffer_size_to = 3
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_from)

to_iter = BufferedIterator((i for i in range(num_items+1, num_items*2)), 
                              buffer_size=buffer_size_to)

fwd_count_from = random.randint(2, buffer_size_from-1)
back_count_from = random.randint(1, fwd_count_from-1)

for i in range(fwd_count_from):
    next(from_iter)
from_iter.backup(back_count_from)

print(buffered_iterator_compare(from_iter, to_iter))

to_iter.link(from_iter)
print(buffered_iterator_compare(from_iter, to_iter))

from_iter.previous_items == to_iter.previous_items
from_iter.future_items == to_iter.future_items 

	                    From Iterator                 To Iterator                   
	Previous Items      [0, 1, 2]                     []                            
	Future Items        [3]                           []                            
	Item Count          3                             0                             
	Step Back           0                             0                             
	Buffer Size         5                             3                             

	                    From Iterator                 To Iterator                   
	Previous Items      [0, 1, 2]                     [0, 1, 2]                     
	Future Items        [3]                           []                            
	Item Count          3                             3                             
	Step Back           0                             0                             
	Buffer Size         5                             3                             



False

|Condition|From BufferedIterator|To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|partial|partial|
|future deque|partial|partial|
|iterator status|in progress|in progress|
|buffer size ||Target smaller|

In [21]:
num_items = 10
buffer_size_from = 5
buffer_size_to = 3
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_from)

to_iter = BufferedIterator((i for i in range(num_items+1, num_items*2)), 
                              buffer_size=buffer_size_to)

fwd_count_from = random.randint(2, buffer_size_from-1)
back_count_from = random.randint(1, fwd_count_from-1)

for i in range(fwd_count_from):
    next(from_iter)
from_iter.backup(back_count_from)

fwd_count_to = random.randint(2, buffer_size_to-1)
back_count_to = random.randint(1, fwd_count_to-1)

for i in range(buffer_size_to + 1):
    next(to_iter)
to_iter.backup(buffer_size_to)

print(buffered_iterator_compare(from_iter, to_iter))

to_iter.link(from_iter)
print(buffered_iterator_compare(from_iter, to_iter))

from_iter.previous_items == to_iter.previous_items
from_iter.future_items == to_iter.future_items 

	                    From Iterator                 To Iterator                   
	Previous Items      [0]                           []                            
	Future Items        [1]                           [12, 13, 14]                  
	Item Count          1                             1                             
	Step Back           0                             0                             
	Buffer Size         5                             3                             

	                    From Iterator                 To Iterator                   
	Previous Items      [0]                           [0]                           
	Future Items        [1]                           []                            
	Item Count          1                             1                             
	Step Back           0                             0                             
	Buffer Size         5                             3                             



False


|Condition|From BufferedIterator |To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|partial|full|
|future deque|partial |full |
|iterator status |in progress |in progress |
|buffer size ||Target larger||

In [22]:
num_items = 10
buffer_size_from = 3
buffer_size_to = 5
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_from)

to_iter = BufferedIterator((i for i in range(num_items+1, num_items*2)), 
                              buffer_size=buffer_size_to)

fwd_count_from = random.randint(2, buffer_size_from-1)
back_count_from = random.randint(1, fwd_count_from-1)

for i in range(fwd_count_from):
    next(from_iter)
from_iter.backup(back_count_from)

fwd_count_to = random.randint(2, buffer_size_to-1)
back_count_to = random.randint(1, fwd_count_to-1)

for i in range(buffer_size_to + 1):
    next(to_iter)
to_iter.backup(buffer_size_to)

print(buffered_iterator_compare(from_iter, to_iter))

to_iter.link(from_iter)
print(buffered_iterator_compare(from_iter, to_iter))

from_iter.previous_items == to_iter.previous_items
from_iter.future_items == to_iter.future_items 

	                    From Iterator                 To Iterator                   
	Previous Items      [0]                           []                            
	Future Items        [1]                           [12, 13, 14, 15, 16]          
	Item Count          1                             1                             
	Step Back           0                             0                             
	Buffer Size         3                             5                             

	                    From Iterator                 To Iterator                   
	Previous Items      [0]                           [0]                           
	Future Items        [1]                           []                            
	Item Count          1                             1                             
	Step Back           0                             0                             
	Buffer Size         3                             5                             



False

#### Iterator Copying

**Note:** After update the two buffered iterators share the same base iterator. Advancing one will advance the other.

|Condition|From BufferedIterator |To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|full|empty|
|future deque|full |empty |
|iterator status |closed |in progress|
|buffer size |Same |Same |
#


In [23]:
num_items = 10
buffer_size_from = 5
buffer_size_to = 5
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_from)

to_iter = BufferedIterator((i for i in range(num_items+1, num_items*2)), 
                              buffer_size=buffer_size_to)

fwd_count_from = num_items
for i in range(fwd_count_from):
    next(from_iter)

print(buffered_iterator_compare(from_iter, to_iter))

to_iter.link(from_iter)
print(buffered_iterator_compare(from_iter, to_iter))

from_iter.previous_items == to_iter.previous_items
from_iter.future_items == to_iter.future_items 

try:
    next(to_iter)
except BufferedIteratorEOF:
    print('pass')

	                    From Iterator                 To Iterator                   
	Previous Items      [5, 6, 7, 8, 9]               []                            
	Future Items        []                            []                            
	Item Count          10                            0                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             

	                    From Iterator                 To Iterator                   
	Previous Items      [5, 6, 7, 8, 9]               [5, 6, 7, 8, 9]               
	Future Items        []                            []                            
	Item Count          10                            10                            
	Step Back           0                             0                             
	Buffer Size         5                             5                             



|Condition|From BufferedIterator |To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|full|full|
|future deque|empty |empty |
|iterator status |closed |closed|
|buffer size |Same |Same |

In [24]:
num_items = 10
buffer_size_from = 5
buffer_size_to = 5
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_from)

to_iter = BufferedIterator((i for i in range(num_items, num_items*2)), 
                              buffer_size=buffer_size_to)

for i in range(num_items):
    next(from_iter)
for i in range(num_items):
    next(to_iter)
    
print(buffered_iterator_compare(from_iter, to_iter))

to_iter.link(from_iter)
print(buffered_iterator_compare(from_iter, to_iter))

from_iter.previous_items == to_iter.previous_items
from_iter.future_items == to_iter.future_items 

try:
    next(to_iter)
except BufferedIteratorEOF:
    print('pass')

	                    From Iterator                 To Iterator                   
	Previous Items      [5, 6, 7, 8, 9]               [15, 16, 17, 18, 19]          
	Future Items        []                            []                            
	Item Count          10                            10                            
	Step Back           0                             0                             
	Buffer Size         5                             5                             

	                    From Iterator                 To Iterator                   
	Previous Items      [5, 6, 7, 8, 9]               [5, 6, 7, 8, 9]               
	Future Items        []                            []                            
	Item Count          10                            10                            
	Step Back           0                             0                             
	Buffer Size         5                             5                             

pass


 
|Condition|From BufferedIterator |To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|empty|empty|
|future deque|empty |empty |
|iterator status |not started |not started |
|buffer size |Same |Same |
#
 


In [25]:
num_items = 10
buffer_size_from = 5
buffer_size_to = 5
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_from)

to_iter = BufferedIterator((i for i in range(num_items, num_items*2)), 
                              buffer_size=buffer_size_to)
to_iter.link(from_iter)
print(list(from_iter))
print(list(to_iter))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [26]:
num_items = 10
buffer_size_from = 5
buffer_size_to = 5
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_from)

to_iter = BufferedIterator((i for i in range(num_items, num_items*2)), 
                              buffer_size=buffer_size_to)

to_iter.link(from_iter)

print(list(to_iter))
print(list(from_iter))

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


|Condition|From BufferedIterator |To BufferedIterator|
|---------|---------------------|-------------------|
|previous deque|full|empty|
|future deque|full |empty |
|iterator status |in progress |not started |
|buffer size |Same |Same |
#


		
### Make BufferedIterator of BufferedIterator

- Verify that:
- step back only impacts top iterator
- queues in both store items from next
- future Items in top are pulled by next without calling inner iterator
- update works to pass future items from top to inner 
- What happens when updating top iterator with sub iterator?

In [27]:
num_items = 10
buffer_size_top = 5
buffer_size_sub = 5

top_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_top)
sub_iter = BufferedIterator(top_iter, buffer_size=buffer_size_sub)

fwd_count_top =   5 # random.randint(2, buffer_size_top-1)
back_count_top =  2 # random.randint(1, fwd_count_top)

for i in range(fwd_count_top):
    next(top_iter)
top_iter.backup(back_count_top)
print(buffered_iterator_compare(top_iter, sub_iter))

fwd_count_sub = 3
back_count_sub = 1

for i in range(fwd_count_sub):
    next(sub_iter)
sub_iter.backup(back_count_sub)

print(buffered_iterator_compare(top_iter, sub_iter))

	                    From Iterator                 To Iterator                   
	Previous Items      [0, 1, 2]                     []                            
	Future Items        [3, 4]                        []                            
	Item Count          3                             0                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             

	                    From Iterator                 To Iterator                   
	Previous Items      [1, 2, 3, 4, 5]               [3, 4]                        
	Future Items        []                            [5]                           
	Item Count          6                             2                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             



#### Test that queues in both store items from next
- Advance Sub Iterator by Top Iterator Buffer Size and back by smaller amount
- Top Iterator Previous Items should contain all items
- Sub Iterator should split items between Previous Items and Future Items.
- Top Iterator Item Count should equal Buffer Size.
- Sub Iterator Item Count should equal Buffer Size - Step Back.

In [28]:
num_items = 10
buffer_size_top = 5
buffer_size_sub = 5

top_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_top)
sub_iter = BufferedIterator(top_iter, buffer_size=buffer_size_sub)

fwd_count_sub = buffer_size_top
back_count_sub = random.randint(1, buffer_size_top-1)

for i in range(fwd_count_sub):
    next(sub_iter)
sub_iter.backup(back_count_sub)

print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))

print(len(top_iter.previous_items) == buffer_size_top)
print(top_iter.previous_items == sub_iter.previous_items + sub_iter.future_items)

	                    Top Iterator                  Sub Iterator                  
	Previous Items      [0, 1, 2, 3, 4]               [0, 1, 2, 3]                  
	Future Items        []                            [4]                           
	Item Count          5                             4                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             

True
True


#### Test that step back in top only impacts top iterator
- Advance Sub Iterator by Top Iterator Buffer Size.
- Backup Top Iterator by smaller amount.
- Top Iterator should split items between Previous Items and Future Items.
- Sub Iterator Previous Items should contain all items.
- Top Iterator Item Count should equal Buffer Size - Step Back.
- Sub Iterator Item Count should equal Buffer Size.

In [29]:
num_items = 10
buffer_size_top = 5
buffer_size_sub = 5

top_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_top)
sub_iter = BufferedIterator(top_iter, buffer_size=buffer_size_sub)

fwd_count_sub =   buffer_size_top
back_count_top =  random.randint(1, fwd_count_sub)

for i in range(fwd_count_sub):
    next(sub_iter)   
top_iter.backup(back_count_top)

print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))

print(len(sub_iter.previous_items) == buffer_size_top)
print(sub_iter.previous_items == top_iter.previous_items + top_iter.future_items)

	                    Top Iterator                  Sub Iterator                  
	Previous Items      [0, 1, 2, 3]                  [0, 1, 2, 3, 4]               
	Future Items        [4]                           []                            
	Item Count          4                             5                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             

True
True


#### Test that future Items in top are pulled by next without calling inner iterator
- Advance Sub Iterator by Top Iterator Buffer Size.
- Backup Top Iterator by smaller amount.
- Advance Top Iterator by even smaller amount.
- Top Iterator should split items between Previous Items and Future Items.
- Sub Iterator Previous Items should contain all items.
- Top Iterator Item Count should equal Buffer Size - Step Back.
- Sub Iterator Item Count should equal Buffer Size.

In [30]:
num_items = 10
buffer_size_top = 5
buffer_size_sub = 5

top_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_top)
sub_iter = BufferedIterator(top_iter, buffer_size=buffer_size_sub)

fwd_count_sub =   buffer_size_top
back_count_top =  random.randint(1, fwd_count_sub)
fwd_count_top =   random.randint(1, back_count_top)

for i in range(fwd_count_sub):
    next(sub_iter)   
top_iter.backup(back_count_top)

print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))
for i in range(fwd_count_top):
    next(top_iter)   
 
print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))
   
print(len(sub_iter.previous_items) == buffer_size_top)
print(sub_iter.previous_items == top_iter.previous_items + top_iter.future_items)

	                    Top Iterator                  Sub Iterator                  
	Previous Items      [0]                           [0, 1, 2, 3, 4]               
	Future Items        [1, 2, 3, 4]                  []                            
	Item Count          1                             5                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             

	                    Top Iterator                  Sub Iterator                  
	Previous Items      [0, 1]                        [0, 1, 2, 3, 4]               
	Future Items        [2, 3, 4]                     []                            
	Item Count          2                             5                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             

True
True


#### Test that update works to pass future items from top to sub 
- Advance Sub Iterator by Top Iterator Buffer Size and back by smaller amount
- Top Iterator Previous Items should contain all items
- Sub Iterator should split items between Previous Items and Future Items.
- Top Iterator Item Count should equal Buffer Size.
- Sub Iterator Item Count should equal Buffer Size - Step Back.

In [31]:
num_items = 10
buffer_size_top = 5
buffer_size_sub = 5

top_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_top)
sub_iter = BufferedIterator(top_iter, buffer_size=buffer_size_sub)

fwd_count_sub =   random.randint(1, buffer_size_sub - 1)
fwd_count_top =   random.randint(1, buffer_size_top - fwd_count_sub)
back_count_top =  random.randint(1, fwd_count_top)

for i in range(fwd_count_sub):
    next(sub_iter)   

for i in range(fwd_count_top):
    next(top_iter) 

top_iter.backup(back_count_top)
next(sub_iter)
print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))


sub_iter.link(top_iter)
print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))

print(len(sub_iter.previous_items) == buffer_size_top)
print(sub_iter.previous_items == top_iter.previous_items + top_iter.future_items)
print(list(sub_iter))

	                    Top Iterator                  Sub Iterator                  
	Previous Items      [0, 1, 2]                     [0, 1, 2]                     
	Future Items        []                            []                            
	Item Count          3                             3                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             

	                    Top Iterator                  Sub Iterator                  
	Previous Items      [0, 1, 2]                     [0, 1, 2]                     
	Future Items        []                            []                            
	Item Count          3                             3                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             

False
True
[3,

### Mixing Sub and top steps can loose items or duplicate

In [32]:
num_items = 10
buffer_size_top = 10
buffer_size_sub = 10

top_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_top)
sub_iter = BufferedIterator(top_iter, buffer_size=buffer_size_sub)

fwd_count_sub = buffer_size_sub - 5
fwd_count_top = buffer_size_top - fwd_count_sub - 3
back_count_top = 4
fwd_count_sub2 = 2

print(f'Advance Sub Iterator by {fwd_count_sub} (Both iterators advance)')
for i in range(fwd_count_sub):
    next(sub_iter)   
print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))

print(f'Advance Top Iterator by {fwd_count_top} (Sub Iterator does not advance).')
for i in range(fwd_count_top):
    next(top_iter) 
print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))

print(f'Backup Top Iterator by {back_count_top} (Sub Iterator does not change).')
top_iter.backup(back_count_top)
print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))

print(f'Advance Sub Iterator by {fwd_count_sub2} (Sub Iterator gets previous items from top iterator).')
for i in range(fwd_count_sub2):
    next(sub_iter)     
print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))

Advance Sub Iterator by 5 (Both iterators advance)
	                    Top Iterator                  Sub Iterator                  
	Previous Items      [0, 1, 2, 3, 4]               [0, 1, 2, 3, 4]               
	Future Items        []                            []                            
	Item Count          5                             5                             
	Step Back           0                             0                             
	Buffer Size         10                            10                            

Advance Top Iterator by 2 (Sub Iterator does not advance).
	                    Top Iterator                  Sub Iterator                  
	Previous Items      [0, 1, 2, 3, 4, 5, 6]         [0, 1, 2, 3, 4]               
	Future Items        []                            []                            
	Item Count          7                             5                             
	Step Back           0                             0                 

### BufferedIterator generators
> **After update the two BufferedIterators share the same generator.**

In [33]:
from_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size)

to_iter = BufferedIterator((i for i in range(num_items+1, num_items*2)), 
                              buffer_size=buffer_size)

for i in range(buffer_size+1):
    next(from_iter)
    next(to_iter)

#print(repr((from_iter)))
#print(repr((to_iter)))

to_iter.link(from_iter)
print(repr((to_iter)))


iter_match = []
while True:
    try:
        i = next(from_iter)
        j = next(to_iter)
    except (StopIteration, BufferedIteratorEOF):
        break
    iter_match.append((i,j))
pprint(iter_match)
print(repr((from_iter)))
print(repr((to_iter)))

BufferedIterator(source=<generator object <genexpr> at 0x0000023E781C9970>, buffer_size=5)
	BufferedIterator.previous_items = deque([1, 2, 3, 4, 5], maxlen=5)
	BufferedIterator.future_items = deque([], maxlen=5)
	BufferedIterator.item_count = 6
	BufferedIterator._step_back = 0
[(6, 17), (7, 18), (8, 19)]
BufferedIterator(source=<generator object <genexpr> at 0x0000023E781A7190>, buffer_size=5)
	BufferedIterator.previous_items = deque([5, 6, 7, 8, 9], maxlen=5)
	BufferedIterator.future_items = deque([], maxlen=5)
	BufferedIterator.item_count = 10
	BufferedIterator._step_back = 0
BufferedIterator(source=<generator object <genexpr> at 0x0000023E781C9970>, buffer_size=5)
	BufferedIterator.previous_items = deque([4, 5, 17, 18, 19], maxlen=5)
	BufferedIterator.future_items = deque([], maxlen=5)
	BufferedIterator.item_count = 9
	BufferedIterator._step_back = 0


### What happens when updating top iterator with sub iterator?
> iterator breaks

In [34]:
num_items = 10
buffer_size_top = 5
buffer_size_sub = 5

top_iter = BufferedIterator((i for i in range(num_items)), 
                              buffer_size=buffer_size_top)
sub_iter = BufferedIterator(top_iter, buffer_size=buffer_size_sub)

fwd_count_sub =   random.randint(1, buffer_size_sub - 1)
fwd_count_top =   random.randint(1, buffer_size_top - fwd_count_sub)
back_count_top =  random.randint(1, fwd_count_top)

for i in range(fwd_count_sub):
    next(sub_iter)   

top_iter.link(sub_iter)
print(buffered_iterator_compare(top_iter, sub_iter, 
                                label1='Top Iterator', 
                                label2='Sub Iterator'))
print(list(sub_iter))

	                    Top Iterator                  Sub Iterator                  
	Previous Items      [0]                           [0]                           
	Future Items        []                            []                            
	Item Count          1                             1                             
	Step Back           0                             0                             
	Buffer Size         5                             5                             

[1, 2, 3, 4, 5, 6, 7, 8, 9]
